In [3]:
import pandas as pd
import csv
import os
import re   
import nltk
from nltk.corpus import stopwords

In [64]:
#Loading data
it_data = pd.read_csv("data/QueryResults.csv")
it_data = it_data.drop('Count', 1)
it_data = it_data.head(2000)
stopWords_fr = set(stopwords.words('french'))

In [65]:
#Check if a word is a technology
def check_word_technology(tech):
    word = get_cleaned_hashtags(tech)
    tech = tech.lower()
    if tech in it_data['TagName'].values:
        return True
    else:
        return False

#get the name of the company
def extract_company_name(email):
    email = email.split('@')
    company_website = email[1].split('.')
    return company_website[0]

#get the list of all included emails in the offer
def get_emails_list(text):
    return re.findall('\S+@\S+', text) 

#clean all hashtags
def get_cleaned_hashtags(text):
    hashtags = re.findall('#\S+', text)
    cleaned = []
    for hashtag in hashtags:
        curr = hashtag[1:]
        cleaned.append(curr)
    return cleaned

def contains_word(s, w):
    return f' {w} ' in f' {s} '

In [66]:
print("-------------Technologies\n")
technologies = set()        
for word in it_data['TagName'].values:
    if contains_word(annonce, word):
        technologies.add(word)

print(technologies)

   
print("\n--------------company: \n")
emails = get_emails_list(annonce)
emails = get_emails_list(annonce)
for email in emails:
    print(extract_company_name(email))

-------------Technologies

{'java', 'dotnet', 'web'}

--------------company: 



## Exploring the files

In [67]:
files = []
path = "offers"
for r, d, f in os.walk(path):
    for file in f:
            files.append(os.path.join(r, file))
len(files)

110

In [68]:
tech_comp = pd.DataFrame(columns=['Technology','Company'])


for file in files:
    f = open(file, "r", encoding="latin-1")
    annonce = f.read()
    
    #Extracting company names
    
    emails = get_emails_list(annonce)
    email_list = set()
    for email in emails:
        email_list.add(extract_company_name(email))
    
    
    if len(email_list) == 0:
        email = "Unknown"
    else:
        email_list = sorted(email_list)
        email = email_list[0]
        
        
    #Extracting Technologies - Removing punctuation - Removing French stopwrds considered as technologies
    
    annonce = annonce.lower()
    tokens = nltk.word_tokenize(annonce)
    annonce = re.sub(r'[^\w\s]','',annonce)
    technologies = set()        
    for word in it_data['TagName'].values:
        if word not in stopWords_fr and contains_word(annonce, word):
            technologies.add(word)
            
    for techno in technologies:
        #print(email, techno)
        tech_comp = tech_comp.append({'Technology': techno,'Company':email}, ignore_index=True)



In [70]:
tech_comp.head()
tech_comp['Technology'].value_counts()

java                 22
web                  21
sap                  14
mobile               12
cloud                12
sql                  11
jee                  11
angular              10
javascript            9
machine learning      8
python                8
php                   8
oracle                7
dotnet                7
devops                7
salesforce            6
html                  5
stack                 5
android               5
product               5
this                  4
server                4
unique                4
email                 4
api                   4
data science          4
blockchain            4
performance           4
excel                 4
cgi                   4
                     ..
computer science      1
algorithm             1
distance              1
swift                 1
powerbi               1
tensorflow            1
transform             1
kotlin                1
excel vba             1
postgresql            1
focus           

In [45]:
company_emails = []
for file in files:
    f = open(file, "r", encoding="latin-1")
    annonce = f.read()
    emails = get_emails_list(annonce)
    email_list = set()
    for email in emails:
        email_list.add(extract_company_name(email))
    sorted(email_list)
    company_emails.append(email_list)
    
for lst in company_emails:
    sorted(lst)
    if len(lst) == 2:
        lst = sorted(lst)
        print(lst[0])

sgats
smarteez
confidentiel
